In [1]:
import scipy.spatial.distance
import pandas as pd
import numpy as np
import re
import os
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from gensim import models
from gensim.corpora import Dictionary
from timeit import default_timer as timer

dictionary = Dictionary()
tfidf_gensim = models.TfidfModel()
model_words = None

def tokenizeText(string):
    #prepares the string for tokenization, to lowercase, then removes punctutation, then changes numbers to _NUM_
    string = string.lower()
    string = re.sub(r"\d+\.?\d*", " _NUM_ ", string)
    string = re.sub(r'[^\w\s]', '',string)
    stop_words = set(stopwords.words("english"))
    tokenized = word_tokenize(string)
    return [word for word in tokenized if not word.lower() in stop_words]

C:\Users\mocka\mambaforge\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
fiveThsnd= r"C:\Users\mocka\PycharmProjects\Patent-Prior-Art-Finder\Patent Queries\5000PatentsAbstractAndCitationsJson.json"
texts = pd.io.json.read_json(fiveThsnd)["Abstract"]
tokens=  texts.apply(tokenizeText)



In [3]:
# makeModel
dictionary.add_documents(tokens)
model_words = Word2Vec(tokens,vector_size=50)
model_words = model_words
model_words.build_vocab(tokens, update=True)
model_words.train(tokens, total_examples=model_words.corpus_count,
                       epochs=model_words.epochs)

(1401012, 1658985)

In [5]:
#def _makeEmbeddings(self, texts):
corpus = [dictionary.doc2bow(line) for line in tokens]
tfidf_gensim = models.TfidfModel(corpus)
tfidfs = [tfidf_gensim[corpus[x]] for x in range(0, len(corpus))]

for (tokenList, tfidfList) in zip(tokens, tfidfs):
    sum_words = np.empty(50)
    tfidfDict = dict(tfidfList)
    passed = []
    for word in tokenList:
        index = dictionary.token2id.get(word)
        tfidfValue = tfidfDict.get(index)
        try:
            print("Old sum: "+str(sum_words))
            print("Add vec:"+str(model_words.wv[word]))
            sum_words= np.add(sum_words ,model_words.wv[word])
            # sum_tfidf= np.add(sum_tfidf,np.multiply(tfidfValue,model_words[word]))
        except: # In case the model does not have a given word, ignore it.
            passed.append(word)
            pass